<a href="https://colab.research.google.com/github/Krykiet/tensorflow-dog-breed-classification/blob/main/dog_visual_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐶 End-to-end Multi-class Dog Breed Classification

Multi-Class image classifier using TensorFlow and TensorFlow Hub.

## 1. Problem

Identifying the breed of a dog given an image of a dog.

## 2. Data

Data is from Kaggle's dog breed identification competition.

https://www.kaggle.com/competitions/dog-breed-identification/data

## 3. Evaluation

The evaluation is a file with prediction probabilities for each dog breed prediction.

## 4. Features

Information about the data:
* Images (unstructured data)
* 120 breeds of dogs (120 classes)
* 10,000+ images in the training set (with labels)
* 10,000+ images in the test set (without labels - to classify)

